In [24]:
import json
import requests
import pandas as pd

In [25]:
df = pd.read_csv('master_cleansing_done.csv')

In [26]:
df

,no,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,slat,slng,endpos1,endpos2,elat,elng
0,7658,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,127.043713,37.614125,동대문구,회기동,127.055313,37.590808
1,7909,중형승합,2019-07-23 07:00:00,2019-07-23 07:56:33,2019-07-23 08:21:30,성북구,장위제1동,127.043713,37.614125,동대문구,회기동,127.055313,37.590808
2,4174,중형승합,2021-08-27 07:00:00,2021-08-27 07:00:27,2021-08-27 07:20:23,성북구,장위제1동,127.043713,37.614125,동대문구,회기동,127.055313,37.590808
3,8040,중형승합,2021-08-12 07:00:00,2021-08-12 07:09:31,2021-08-12 07:18:53,성북구,장위제1동,127.043713,37.614125,동대문구,회기동,127.055313,37.590808
4,8279,중형승합,2022-03-02 07:22:00,2022-03-02 07:27:35,2022-03-02 07:55:29,성북구,장위제1동,127.043713,37.614125,동대문구,회기동,127.055313,37.590808
...,...,...,...,...,...,...,...,...,...,...,...,...,...
389145,3600,중형승합,2020-12-25 08:07:00,2020-12-25 08:08:26,2020-12-25 09:29:43,성북구,길음제2동,127.027316,37.606665,수정구,신흥1동,127.141172,37.443038
389146,4964,중형승합,2021-05-16 09:14:03,2021-05-16 09:22:04,2021-05-16 09:54:56,성북구,길음제2동,127.027316,37.606665,수정구,신흥1동,127.141172,37.443038
389147,1192,중형승합,2018-09-25 08:36:00,2018-09-25 08:41:28,2018-09-25 09:12:20,관악구,행운동,126.957046,37.480654,기흥구,신갈동,127.107056,37.272653
389148,1556,중형승합,2018-06-16 07:00:00,2018-06-16 07:07:27,2018-06-16 07:32:12,강북구,인수동,127.010631,37.641511,분당구,서현2동,127.134599,37.376037


In [27]:
dummy = df[['startpos2', 'endpos2', 'slat', 'slng', 'elat', 'elng']]

In [28]:
dummy.columns = ['start', 'end', 'startlat', 'startlng', 'endlat', 'endlng']

In [29]:
# startpos2와 endpos2를 붙여 고유한 것을 가져와 df_new로 만듭니다.
a = []
for i in range(len(dummy)):
    a.append(dummy['start'][i] + dummy['end'][i])
dummy['direction'] = a
df_new = dummy.drop_duplicates()
df_new = df_new.reset_index(drop = True)
df_new

/var/folders/3q/y966c1fj3lndmps8kk3plpy40000gn/T/ipykernel_4619/4177500277.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy['direction'] = a


,start,end,startlat,startlng,endlat,endlng,direction
0,장위제1동,회기동,127.043713,37.614125,127.055313,37.590808,장위제1동회기동
1,고척제2동,회기동,126.858712,37.506706,127.055313,37.590808,고척제2동회기동
2,회기동,회기동,127.055313,37.590808,127.055313,37.590808,회기동회기동
3,상봉제1동,회기동,127.087200,37.599880,127.055313,37.590808,상봉제1동회기동
4,구의제1동,회기동,127.085676,37.542492,127.055313,37.590808,구의제1동회기동
...,...,...,...,...,...,...,...
30700,방화제1동,신흥1동,126.817026,37.570366,127.141172,37.443038,방화제1동신흥1동
30701,길음제2동,신흥1동,127.027316,37.606665,127.141172,37.443038,길음제2동신흥1동
30702,행운동,신갈동,126.957046,37.480654,127.107056,37.272653,행운동신갈동
30703,인수동,서현2동,127.010631,37.641511,127.134599,37.376037,인수동서현2동


In [30]:
# direction 5 와 direction 15 API 키를 각각 생성합니다. 각 api는 하루 6000건이 최대입니다.

# 총 4명이므로, 8개의 api를 생성할 수 있습니다. df_new의 개수는 32518개이므로, df_new를 4100개씩 나눠 진행합니다.
# 각 데이터프레임을 각각 api 돌린 후, 다 뽑아내오면 하나로 합쳐줍니다.

# 하나로 합쳐진 데이터프레임 df_new는 direction의 고유값이기 때문에, 이후 df에 join merge 시키면 됩니다.

df_new3 = df_new[8200:12300].reset_index(drop=True) # 은지(direction 5 사용)
df_new4 = df_new[12300:16400].reset_index(drop=True) # 은지(direction 15 사용)

In [32]:
len(df_new3) ,len(df_new4)

(4100, 4100)

# 여기서부터 조심히 사용해주세요. api를 사용합니다.

# Direction 5 API와 Direction 15 API 수정하세요.

In [33]:
# direction 5 API에 대한 ID, Key 입력
dir_5_ID = "k4y8ulq5wf"
dir_5_Key = "oxApuodzfOi5jjgahPylNK0b3mTPs8YHxB9lwBbD"

# direction 15 API에 대한 ID, Key 입력
dir_15_ID = "k4y8ulq5wf"
dir_15_Key = "oxApuodzfOi5jjgahPylNK0b3mTPs8YHxB9lwBbD"

In [34]:
## 여기는 수정하지 마세요!

# 헤더 설정
headers_5 = {'X-NCP-APIGW-API-KEY-ID' : dir_5_ID, 'X-NCP-APIGW-API-KEY' : dir_5_Key}
headers_15 = {'X-NCP-APIGW-API-KEY-ID' : dir_15_ID, 'X-NCP-APIGW-API-KEY' : dir_15_Key}

In [35]:
distance = []
duration = []
taxiFare = []

"""
여기부터는 df_new 자기 숫자에 맞춰 진행하세요.
"""
# 주의! 여기서부터 api를 사용합니다. headers_5 먼저 돌립니다.
# for > range > len 안의 값과 url 내의 변수를 꼭 변경해주세요.

for i in range(len(df_new3)):
    slat = df_new3['startlat'][i]
    slng = df_new3['startlng'][i]
    elat = df_new3['endlat'][i]
    elng = df_new3['endlng'][i]
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={slat},{slng}&goal={elat},{elng}"
    try:
        document = requests.get(url, headers=headers_5).json()
        distance.append(document['route']['traoptimal'][0]['summary']['distance'])
        duration.append(document['route']['traoptimal'][0]['summary']['duration'] / 1000)
        taxiFare.append(document['route']['traoptimal'][0]['summary']['taxiFare'])
    except:
        distance.append(0)
        duration.append(0)
        taxiFare.append(0)
        
# df_new에 각 변수를 추가하여 넣습니다.
# df_new 숫자만 변경해서 쓰세요
df_new3['distance'] = distance
df_new3['duration'] = duration
df_new3['taxiFare'] = taxiFare

# csv로 저장합니다.
df_new3.to_csv('df_new3.csv', encoding='utf-8')

In [36]:
# 기타 설정
distance = []
duration = []
taxiFare = []

"""
여기부터는 df_new 자기 숫자에 맞춰 진행하세요.
"""
# 주의! api를 사용합니다. headers_15 돌립니다.
# for > range > len 안의 값과 url 내의 변수를 꼭 변경해주세요. 
for i in range(len(df_new4)):
    slat = df_new4['startlat'][i]
    slng = df_new4['startlng'][i]
    elat = df_new4['endlat'][i]
    elng = df_new4['endlng'][i]
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={slat},{slng}&goal={elat},{elng}"
    try:
        document = requests.get(url, headers=headers_15).json()
        distance.append(document['route']['traoptimal'][0]['summary']['distance'])
        duration.append(document['route']['traoptimal'][0]['summary']['duration'] / 1000)
        taxiFare.append(document['route']['traoptimal'][0]['summary']['taxiFare'])
    except:
        distance.append(0)
        duration.append(0)
        taxiFare.append(0)

# df_new에 각 변수를 추가하여 넣습니다.
# df_new 숫자만 변경해서 쓰세요
df_new4['distance'] = distance
df_new4['duration'] = duration
df_new4['taxiFare'] = taxiFare

# csv로 저장합니다.
df_new4.to_csv('df_new4.csv', encoding='utf-8')

,start,end,startlat,startlng,endlat,endlng,direction,distance,duration,taxiFare
0,장위제3동,명동,127.056603,37.616551,126.985850,37.560042,장위제3동명동,12401,1825.466,13000
1,혜화동,명동,127.000576,37.586919,126.985850,37.560042,혜화동명동,5280,894.299,7000
2,망원제1동,명동,126.905597,37.555622,126.985850,37.560042,망원제1동명동,8771,1603.297,9900
3,천연동,명동,126.959033,37.571125,126.985850,37.560042,천연동명동,4041,727.247,5300
4,용산2가동,명동,126.985578,37.546133,126.985850,37.560042,용산2가동명동,2501,410.871,4500
...,...,...,...,...,...,...,...,...,...,...
4095,영등포본동,청량리동,126.909305,37.514683,127.047277,37.586259,영등포본동청량리동,22007,2415.884,20000
4096,이문제1동,청량리동,127.065450,37.597849,127.047277,37.586259,이문제1동청량리동,3847,522.861,5300
4097,전농제1동,청량리동,127.047758,37.577998,127.047277,37.586259,전농제1동청량리동,2353,546.476,4000
4098,고양동,청량리동,126.903703,37.703366,127.047277,37.586259,고양동청량리동,33631,2475.205,33050


In [40]:
df_new4

,start,end,startlat,startlng,endlat,endlng,direction,distance,duration,taxiFare
0,장위제3동,명동,127.056603,37.616551,126.985850,37.560042,장위제3동명동,12401,1825.466,13000
1,혜화동,명동,127.000576,37.586919,126.985850,37.560042,혜화동명동,5280,894.299,7000
2,망원제1동,명동,126.905597,37.555622,126.985850,37.560042,망원제1동명동,8771,1603.297,9900
3,천연동,명동,126.959033,37.571125,126.985850,37.560042,천연동명동,4041,727.247,5300
4,용산2가동,명동,126.985578,37.546133,126.985850,37.560042,용산2가동명동,2501,410.871,4500
...,...,...,...,...,...,...,...,...,...,...
4095,영등포본동,청량리동,126.909305,37.514683,127.047277,37.586259,영등포본동청량리동,22007,2415.884,20000
4096,이문제1동,청량리동,127.065450,37.597849,127.047277,37.586259,이문제1동청량리동,3847,522.861,5300
4097,전농제1동,청량리동,127.047758,37.577998,127.047277,37.586259,전농제1동청량리동,2353,546.476,4000
4098,고양동,청량리동,126.903703,37.703366,127.047277,37.586259,고양동청량리동,33631,2475.205,33050
